<a href="https://colab.research.google.com/github/HappyCocker1111/IrisClassification/blob/main/%E6%A9%9F%E6%A2%B0%E5%AD%A6%E7%BF%92%E3%83%A2%E3%83%87%E3%83%AB%E3%81%AE%E7%94%9F%E6%88%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# データの準備

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# 学習用データの読み込み
train_data = pd.read_csv('train.tsv', sep='\t')
train_data.head()

,id,sepal length in cm,sepal width in cm,petal length in cm,petal width in cm,class
0,0,5.3,3.7,1.5,0.2,Iris-setosa
1,1,6.8,2.8,4.8,1.4,Iris-versicolor
2,3,6.1,3.0,4.9,1.8,Iris-virginica
3,4,6.4,3.2,5.3,2.3,Iris-virginica
4,5,6.3,3.3,4.7,1.6,Iris-versicolor


In [2]:
# データフレームのデータ型を表示
print(train_data.dtypes)

id                      int64
sepal length in cm    float64
sepal width in cm     float64
petal length in cm    float64
petal width in cm     float64
class                  object
dtype: object


In [3]:
# ラベルエンコーディング
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train_data['class'] = le.fit_transform(train_data['class'])

In [4]:
train_data.head()

,id,sepal length in cm,sepal width in cm,petal length in cm,petal width in cm,class
0,0,5.3,3.7,1.5,0.2,0
1,1,6.8,2.8,4.8,1.4,1
2,3,6.1,3.0,4.9,1.8,2
3,4,6.4,3.2,5.3,2.3,2
4,5,6.3,3.3,4.7,1.6,1


# K近傍分類器を作成

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

# 特徴量と目的変数の抽出
X = train_data[['sepal length in cm', 'sepal width in cm', 'petal length in cm', 'petal width in cm']]
y = train_data['class']

# 分割

In [6]:
# テストデータの分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 学習

In [7]:
# K近傍分類器を作成
knn = KNeighborsClassifier(n_neighbors=5)

# モデルの学習
knn.fit(X_train, y_train)

# テストデータで予測
y_pred = knn.predict(X_test)

# 評価

In [8]:
# 予測結果と正解ラベルの比較
print(pd.DataFrame({'予測': y_pred, '正解': y_test}))

    予測  正解
4    1   1
63   0   0
10   1   1
0    0   0
35   2   2
61   0   0
28   2   2
12   2   2
69   2   2
64   0   0
34   0   0
42   0   0
49   2   2
18   1   1
9    2   2


In [9]:
accuracy = (y_pred == y_test).mean()
print("正解率:", accuracy)

正解率: 1.0


In [10]:
error_rate = 1 - accuracy
print("錯誤率:", error_rate)

錯誤率: 0.0


今回は100％の確率で正解を当てることができたので、非常に有用なモデルができたといえる。

In [11]:
from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(y_test, y_pred)
print(confusion)

[[6 0 0]
 [0 3 0]
 [0 0 6]]


このK近傍法モデルは、3つのクラスをすべて100%の精度で正しく識別することができているため、非常に高い性能を発揮していると言える。


今回はいずれも100％の精度で識別できており、よいモデルといえると思われるがこれは3つのクラスがバランスよく配置されているからともとらえることができるため過信することは禁物である。


モデルの汎化性能を評価するために、交差検証などの手法を用いて、様々なデータセットでモデルの精度を検証することが重要といえる。

In [12]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
import matplotlib.pyplot as plt

In [13]:
from sklearn.metrics import f1_score

f1_macro = f1_score(y_test, y_pred, average='macro')
print("F1スコア (macro):", f1_macro)

f1_weighted = f1_score(y_test, y_pred, average='weighted')
print("F1スコア (weighted):", f1_weighted)



F1スコア (macro): 1.0
F1スコア (weighted): 1.0


今回、異なるクラスのサンプル数や重要度を考慮した指標を用いて検証を行った。いずれも、1.0という高い値を示した。そのため、モデルが訓練データのすべてのサンプルを過度に学習し、未知データに対しうまく一般化できない可能性が高いことを示唆している。

対策：

* モデルの複雑度を下げる：特徴量の数やニューロンの数を減らしたり、正則化手法を適応したりすることが有用

* データを増やす：訓練データを増やすことでモデルがより多くのパターンを学習し一般化能力を向上させることができる

* 異なるデータセットで学習する：訓練データとは異なるデータセットでモデルを評価することで、過学習の程度をより客観的に判断することができる

# 過学習が行われているか検証をする


In [3]:
# 必要なライブラリをインポート
import pandas as pd
import plotly.express as px
from sklearn.model_selection import train_test_split

# 学習用データの読み込み
train_data = pd.read_csv('train.tsv', sep='\t')

# データの確認
print(train_data.head())

# 3D散布図を作成
fig = px.scatter_3d(train_data,
                    x='sepal length in cm',
                    y='sepal width in cm',
                    z='petal length in cm',
                    color='class',
                    title="3D Scatter Plot of Iris Dataset",
                    labels={
                        'sepal length in cm': 'Sepal Length (cm)',
                        'sepal width in cm': 'Sepal Width (cm)',
                        'petal length in cm': 'Petal Length (cm)'
                    })

# グラフを表示
fig.show()

   id  sepal length in cm  sepal width in cm  petal length in cm  \
0   0                 5.3                3.7                 1.5   
1   1                 6.8                2.8                 4.8   
2   3                 6.1                3.0                 4.9   
3   4                 6.4                3.2                 5.3   
4   5                 6.3                3.3                 4.7   

   petal width in cm            class  
0                0.2      Iris-setosa  
1                1.4  Iris-versicolor  
2                1.8   Iris-virginica  
3                2.3   Iris-virginica  
4                1.6  Iris-versicolor  


３D可視化した結果きれいにデータが分離されていることがわかる